# Get cantons and coordinates for universities

Import libraries. Here we utilize a very useful libarary `requests_cache`, which basically stores your every http requests results into a local database, default is sqlite, in order to **eliminate redundant requests**. We can use store the response for further uses because we are querying **static information**, such as locations for a place, meaning the returned results won't change during further requests.

In [4]:
import pandas as pd
import numpy as np
import requests
import requests_cache
import csv

from collections import Counter


requests_cache.install_cache('ada_cache')

Notice the csv file uses **semicolon** as its delimiter

In [2]:
csv_data = pd.read_csv('P3_GrantExport.csv', sep=';')

Extract out the two columns we are concerned about

In [3]:
data = csv_data[['University', 'Approved Amount']]

In [4]:
data_list = data.to_dict('records')  # change dataframe to a dict

In [5]:
KEY = XXX  # omit my key
GEOCODE_URL = 'https://maps.googleapis.com/maps/api/geocode/json'
TEXT_URL = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

In [6]:
def get_university_amount(data_list):
    """
    find the total funding amount for a university 
    
    params:
        data_list (list): a list of dicts containing university name and amount for one project
        
    return:
        universities (dict): contains university name, total amount of funding, 
                             canton of the university, latitude and longitude
        cnt (Counter): count how many times we missed the data for one university
    """
    cnt = Counter()
    universities = {}
    for d in data_list:
        if isinstance(d['University'], str) and '.' in d['Approved Amount']:
            university, amount = d['University'], float(d['Approved Amount'])
            uni_fullname = university.split(' - ')[0]
            if uni_fullname in universities:
                universities[uni_fullname]['amount'] += amount
            else:  # get university location info etco
                text_params = {
                    'key': KEY,
                    'query': uni_fullname
                }
                text_res = requests.get(TEXT_URL, params=text_params).json()  # we use google json API
                if text_res['status'] == 'OVER_QUERY_LIMIT':
                    with open('universities.csv', 'w') as f:
                        writer = csv.writer(f)
                        for university, info in universities.items():
                            writer.writerow([university, 
                                             "{0:.2f}".format(info['amount']), 
                                             info['canton'], 
                                             info['location']['lat'], 
                                             info['location']['lng']
                                            ])
                    return universities
                elif text_res['status'] != 'OK':
                    cnt[university] += 1
                else:  # get address of text query
                    address = text_res['results'][0]['formatted_address']  # get the first formatted address
                    # send second request to get canton, location etc.
                    geo_params = {
                        'key': KEY,
                        'address': address
                    }
                    geocode_res = requests.get(GEOCODE_URL, params=geo_params).json()
                    first_res = geocode_res['results'][0]   # get the first result
                    if 'geometry' in first_res:
                        geometry = first_res['geometry']
                        if 'location' not in geometry:
                            cnt['no_location'] += 1
                        else:
                            universities[uni_fullname] = {
                                'location': geometry['location'],
                                'amount': amount,
                                'canton': None
                            }  # add a new university
                            addr_comp = first_res['address_components']
                            for addr in addr_comp:
                                if addr['types'][0] == 'administrative_area_level_1':
                                    universities[uni_fullname]['canton'] = addr['long_name']   #  update canton
                    else:
                        cnt['no_geometry'] += 1
        else:
            cnt['not_university'] += 1

    return universities, cnt

uni, c = get_university_amount(data_list)

Then we write a function to write the results into a csv file for futher analysis

In [7]:
def res_to_csv(universities, filename):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['university', 'amount', 'canton', 'latitude', 'longitude'])
        for university, info in universities.items():
            writer.writerow([university, 
                             "{0:.2f}".format(info['amount']), 
                             info['canton'], 
                             info['location']['lat'], 
                             info['location']['lng']
                            ])
res_to_csv(uni, 'universities.csv')

We can check the returned Counter object to see the universities whose information we can't get by Google API. The values are the number of their appearances in the data

In [8]:
c

Counter({'Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL': 223,
         'Eidg. Material und Prüfungsanstalt - EMPA': 238,
         'Fachhochschule Nordwestschweiz (ohne PH) - FHNW': 225,
         'Firmen/Privatwirtschaft - FP': 492,
         'Forschungsanstalten Agroscope - AGS': 135,
         'Forschungskommission SAGW': 1,
         'Haute école pédagogique BE, JU, NE - HEPBEJUNE': 7,
         'NPO (Biblioth., Museen, Verwalt.) - NPO': 1473,
         'Nicht zuteilbar - NA': 2595,
         'Physikal.-Meteorolog. Observatorium Davos - PMOD': 48,
         'Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP': 2,
         'Schweizer Kompetenzzentrum Sozialwissensch. - FORS': 30,
         'Staatsunabh. Theologische Hochschule Basel - STHB': 3,
         'Swiss Center for Electronics and Microtech. - CSEM': 28,
         'Swiss Institute of Bioinformatics - SIB': 31,
         'Weitere Institute - FINST': 43,
         'Weitere Spitäler - ASPIT': 81,
         'not_university': 13091})

Considering there are only a few missed universities, we decided to add them **manually**. Notice there are some invalid university names listed in the following:

1. Firmen/Privatwirtschaft - FP, meaning private in Germany
2. NPO (Biblioth., Museen, Verwalt.) - NPO, meaning Non-profit organization
3. Nicht zuteilbar - NA, meaning not assgined in Germany
4. Weitere Institute - FINST, meaning other institues in Germany
5. Weitere Spitäler - ASPIT, meaning other hospitals in Germany
6. not_university, meaning there's no university name or amount for this project in P3 data

so we only add the rest. Plus, we find **Pädag. Hochschule Tessin** and **Forschungsanstalten Agroscope** have multiple locations and we don't know which office got the fund, so we just ignore these two.

We use the following function to calculate the total amount for each missing university

In [9]:
def generate_record(university):
    """
    get amount for this university
    
    params:
        university (str): universitye name
        
    return:
        a tuple: university and its amount
    """
    university_df = data[data.University == university].copy()
    university_df = university_df[university_df['Approved Amount'] != 'data not included in P3']
    university_df['amount'] = university_df['Approved Amount'].astype('float')

    return university, "{0:.2f}".format(university_df['amount'].sum())

In [10]:
for name in c.keys():
    print(generate_record(name))

('Forschungskommission SAGW', '100000.00')
('Swiss Center for Electronics and Microtech. - CSEM', '18068246.00')
('Schweizer Kompetenzzentrum Sozialwissensch. - FORS', '34735816.00')
('Haute école pédagogique BE, JU, NE - HEPBEJUNE', '627380.00')
('Fachhochschule Nordwestschweiz (ohne PH) - FHNW', '42771914.12')
('Physikal.-Meteorolog. Observatorium Davos - PMOD', '12098436.00')
('Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL', '48360389.63')
('Nicht zuteilbar - NA', '142425719.57')
('Weitere Spitäler - ASPIT', '10749808.00')
('not_university', '0.00')
('Weitere Institute - FINST', '9256736.00')
('Staatsunabh. Theologische Hochschule Basel - STHB', '17300.00')
('Swiss Institute of Bioinformatics - SIB', '11583219.00')
('NPO (Biblioth., Museen, Verwalt.) - NPO', '334130583.79')
('Eidg. Material und Prüfungsanstalt - EMPA', '58574515.92')
('Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP', '159317.00')
('Forschungsanstalten Agroscope - AGS', '33115719.00')
('Firmen/Privatwirtschaf

So finally we have a result csv containing **70 unique universities** of **46058 records** from original P3 data.

Before we start plotting, we check the unique cantons we get and its number.

In [5]:
results = pd.read_csv('results.csv')
results['canton'].value_counts()

Zürich          10
Vaud             7
Bern             6
Ticino           6
Valais           6
Graubünden       5
St. Gallen       5
Luzern           4
Basel-Stadt      3
Neuchâtel        3
Thurgau          2
Solothurn        2
Fribourg         2
Aargau           2
Schwyz           1
Geneve           1
Schaffhausen     1
Zug              1
Geneva           1
Jura             1
Name: canton, dtype: int64

Then we noticed not all canton names returned by Google are in English, some are in Germany and some in French. So we decided to use a predefined map to aggregate same canton. We also deal with the case that the info of two universities returned by Google API were not in Switzerland. In such cases, we then checked the Institutions of these two universities and found their corresponding Swiss cantons, or just remove the record if we couldn't find the Swiss institution, e.g. we remove records for university Istituto Svizzero di Roma, which is said to be in Lazio, Italy according to Google.

In [6]:
CANTON_MAP = {
    'Sankt Gallen': 'St. Gallen',
    'Wallis': 'Valais',
    'Genève': 'Geneve',
    'Geneva': 'Geneve',
    'Baden-Württemberg': 'Valais',
    'Hessen': 'Aargau'
}

In [7]:
def map_canton(x):
    if x in CANTON_MAP:
        return CANTON_MAP[x]
    return x

results['canton'] = results['canton'].apply(map_canton)
results = results[results.canton != 'Lazio']

In [8]:
results['canton'].value_counts()

Zürich          10
Vaud             7
Bern             6
Ticino           6
Valais           6
Graubünden       5
St. Gallen       5
Luzern           4
Basel-Stadt      3
Neuchâtel        3
Geneve           2
Thurgau          2
Solothurn        2
Fribourg         2
Aargau           2
Schwyz           1
Schaffhausen     1
Zug              1
Jura             1
Name: canton, dtype: int64

**Write our final results to the final csv for futher analysis**

In [10]:
results.to_csv('results.csv', index=False)